In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys; sys.path.append('..')
import random, math
import pyzx as zx
from fractions import Fraction
from pyzx.phasepoly import circuit_phase_polynomial_blocks, optimize_block
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\scala.jar'

In [15]:
def generate_parities(targets):
    parities = set()
    for i in range(4):
        a = targets[i]
        parities.add(frozenset([a]))
        for j in range(i+1,4):
            b = targets[j]
            parities.add(frozenset([a,b]))
            for k in range(j+1,4):
                c = targets[k]
                parities.add(frozenset([a,b,c]))
                for l in range(k+1,4):
                    d = targets[l]
                    parities.add(frozenset([a,b,c,d]))
    return parities

s1 = generate_parities([1,2,3,4])
s2 = generate_parities([2,3,4,5])
s3 = generate_parities([1,3,4,5])
s4 = generate_parities([1,2,4,5])
s5 = generate_parities([1,2,3,5])
s1.symmetric_difference(s2).symmetric_difference(s3).symmetric_difference(s4)#.symmetric_difference(s5)

{frozenset({2, 4}),
 frozenset({5}),
 frozenset({3}),
 frozenset({1}),
 frozenset({1, 3, 5}),
 frozenset({1, 2, 3, 4}),
 frozenset({2, 3, 5}),
 frozenset({1, 4}),
 frozenset({1, 2, 5}),
 frozenset({3, 4}),
 frozenset({1, 3, 4, 5}),
 frozenset({2}),
 frozenset({2, 3, 4, 5}),
 frozenset({1, 2, 3}),
 frozenset({4, 5}),
 frozenset({1, 2, 4, 5})}

In [3]:
data = """.v a,b,c,d,e,f
.i a,b,c,d,e,f
.o a,b,c,d,e,f
BEGIN
t3 c,e,d
t2 c,e
t2 d,f
t1 a
t2 f,e
t3 e,f,d
t4 c,d,f,e
t1 d
t3 d,e,f
t3 e,f,d
t2 c,e
t3 a,f,c
t2 e,c
t3 a,c,b
t4 b,d,f,c
t2 a,b
t4 a,b,f,c
t1 b
t5 a,b,d,e,f
t1 b
t2 c,a
t3 c,d,b
t4 b,c,d,a
t3 c,d,b
t3 c,f,a
t3 c,f,b
t3 b,f,c
t4 a,b,f,c
t2 f,b
t2 d,f
t2 f,d
t1 a
t5 a,b,c,d,e
t1 a
t3 d,e,f
t3 b,c,d
t2 d,f
t3 d,e,f
t4 a,c,f,b
t5 a,b,c,e,f
t1 b
t5 b,c,e,f,a
t1 b
t3 b,d,a
t2 e,c
t5 b,c,d,f,a
t2 e,c
t4 b,c,e,f
t2 b,c
t3 a,d,c
t2 d,f
t6 a,b,c,d,e,f
t3 b,f,a
t3 a,b,c
t3 b,f,c
t2 d,a
t3 d,e,f
t3 b,d,a
t6 a,b,d,e,f,c
t1 b
t1 f
END"""

In [6]:
from pyzx.circuit import Z, S, T, CNOT, Tofolli, HAD, NOT

preamble = data[:data.find("BEGIN")].strip().splitlines()
labels = {}
for l in preamble:
    if l.startswith('#'):continue
    if l.startswith('.'):
        for v in l[2:].split(','):
            s = v.strip()
            if s not in labels: labels[s] = len(labels)
    else:
        raise TypeError("Unknown Expression: " + l)

ancillas = {}
gates = []
qcount = len(labels)

for l in data[data.find("BEGIN")+6:data.find("END")].splitlines():
    gname, targets = l.split(' ',1)
    gname = gname.strip().lower()
    targets = [labels[v.strip()] for v in targets.split(',') if v.strip()]
    if len(targets) == 1:
        t = targets[0]
        if gname in ('tof', 't1', 'not', 'x'): gates.append(NOT(t))
        elif gname == 'z': gates.append(Z(t))
        elif gname == 's': gates.append(S(t))
        elif gname == 's*': gates.append(S(t,adjoint=True))
        elif gname == 't': gates.append(T(t))
        elif gname == 't*': gates.append(T(t,adjoint=True))
        elif gname == 'h': gates.append(HAD(t))
        else:
            raise TypeError("Unknown gate with single target: " + l)
    elif len(targets) == 2:
        if gname in ('cnot', 'tof', 't2'):
            c,t = targets
            gates.append(CNOT(c,t))
        else:
            raise TypeError("Unknown gate with control: " + l)
    elif len(targets) == 3:
        if gname in ('t3', 'tof'):
            c1,c2,t = targets
            gates.append(Tofolli(c1,c2,t))
        else:
            raise TypeError("Unknown gate with control: " + l)
    else:
        if gname not in ('t4', 't5', 't6', 'tof'):
            raise TypeError("Unknown gate with multiple controls: " + l)
        *ctrls, t = targets
        if len(ctrls) > 5: raise TypeError("No more than 5 ctrls supported")
        while len(ancillas) < len(ctrls) - 2:
            ancillas[len(ancillas)] = qcount
            qcount += 1
        gates.append(Tofolli(ctrls[0],ctrls[1],ancillas[0]))
        if len(ctrls) == 3:
            gates.append(Tofolli(ctrls[2],ancillas[0],t))
        else:
            gates.append(Tofolli(ctrls[2],ctrls[3],ancillas[1]))
            if len(ctrls) == 4:
                gates.append(Tofolli(ancillas[0],ancillas[1],t))
            else:
                gates.append(Tofolli(ancillas[0],ancillas[1],ancillas[2]))
                gates.append(Tofolli(ctrls[4],ancillas[2],t))
                gates.append(Tofolli(ancillas[0],ancillas[1],ancillas[2]))
            gates.append(Tofolli(ctrls[2],ctrls[3],ancillas[1]))
        gates.append(Tofolli(ctrls[0],ctrls[1],ancillas[0]))

c = zx.Circuit(qcount)
c.gates = gates
c.gates

[Tof(c1=2,c2=4,t=3),
 CNOT(2,4),
 CNOT(3,5),
 NOT(0),
 CNOT(5,4),
 Tof(c1=4,c2=5,t=3),
 Tof(c1=2,c2=3,t=6),
 Tof(c1=5,c2=6,t=4),
 Tof(c1=2,c2=3,t=6),
 NOT(3),
 Tof(c1=3,c2=4,t=5),
 Tof(c1=4,c2=5,t=3),
 CNOT(2,4),
 Tof(c1=0,c2=5,t=2),
 CNOT(4,2),
 Tof(c1=0,c2=2,t=1),
 Tof(c1=1,c2=3,t=6),
 Tof(c1=5,c2=6,t=2),
 Tof(c1=1,c2=3,t=6),
 CNOT(0,1),
 Tof(c1=0,c2=1,t=6),
 Tof(c1=5,c2=6,t=2),
 Tof(c1=0,c2=1,t=6),
 NOT(1),
 Tof(c1=0,c2=1,t=6),
 Tof(c1=3,c2=4,t=7),
 Tof(c1=6,c2=7,t=5),
 Tof(c1=3,c2=4,t=7),
 Tof(c1=0,c2=1,t=6),
 NOT(1),
 CNOT(2,0),
 Tof(c1=2,c2=3,t=1),
 Tof(c1=1,c2=2,t=6),
 Tof(c1=3,c2=6,t=0),
 Tof(c1=1,c2=2,t=6),
 Tof(c1=2,c2=3,t=1),
 Tof(c1=2,c2=5,t=0),
 Tof(c1=2,c2=5,t=1),
 Tof(c1=1,c2=5,t=2),
 Tof(c1=0,c2=1,t=6),
 Tof(c1=5,c2=6,t=2),
 Tof(c1=0,c2=1,t=6),
 CNOT(5,1),
 CNOT(3,5),
 CNOT(5,3),
 NOT(0),
 Tof(c1=0,c2=1,t=6),
 Tof(c1=2,c2=3,t=7),
 Tof(c1=6,c2=7,t=4),
 Tof(c1=2,c2=3,t=7),
 Tof(c1=0,c2=1,t=6),
 NOT(0),
 Tof(c1=3,c2=4,t=5),
 Tof(c1=1,c2=2,t=3),
 CNOT(3,5),
 Tof(c1=3,c2=4,t

In [9]:
g = c.to_graph()
print(zx.tcount(g))
zx.full_reduce(g, quiet=False)
g.normalise()
zx.tcount(g)

567
spider_simp: 206. 169. 101. 57. 37. 24. 8. 3. 1.  9 iterations
id_simp: 88. 29.  2 iterations
spider_simp: 26.  1 iterations
pivot_simp: 26. 1.  2 iterations
lcomp_simp: 15. 15. 2. 2.  4 iterations
id_simp: 3.  1 iterations
spider_simp: 2. 1.  2 iterations
pivot_simp: 1.  1 iterations
lcomp_simp: 2. 2.  2 iterations
Gadgetizing...
pivot_simp: 173. 104. 52. 37. 30. 14. 6. 9. 9. 7. 4. 3. 4. 2. 3. 2. 2. 2. 1.  19 iterations
Back to clifford_simp
id_simp: 13.  1 iterations
spider_simp: 6. 3. 2. 2.  4 iterations
pivot_simp: 6. 6. 5. 6. 5. 4. 7. 4. 3. 2.  10 iterations
id_simp: 36.  1 iterations
spider_simp: 14. 8. 6. 4. 3. 1.  6 iterations
pivot_simp: 3. 2.  2 iterations
lcomp_simp: 12. 13. 3. 3. 2. 2. 1.  7 iterations
id_simp: 19.  1 iterations
spider_simp: 5. 2. 2. 2. 2. 2. 1. 2.  8 iterations
pivot_simp: 3.  1 iterations
lcomp_simp: 3. 3.  2 iterations
id_simp: 2.  1 iterations
spider_simp: 2.  1 iterations
pivot_simp: 1.  1 iterations
lcomp_simp: 2. 2.  2 iterations
id_simp: 1.  1 i

279